In [1]:
from pathlib import Path
from datetime import datetime
import random, time, csv, json, math, statistics

BASE = Path('./tmp/stage14_demo')
BASE.mkdir(parents=True, exist_ok=True)
METRICS_CSV = BASE / 'metrics.csv'
if not METRICS_CSV.exists():
    with METRICS_CSV.open('w', newline='') as f:
        w = csv.writer(f)
        w.writerow(['ts','name','value','layer','model_version','window','context'])
print(f'Writing metrics to: {METRICS_CSV}')

Writing metrics to: tmp\stage14_demo\metrics.csv


In [2]:
def log_metric(name, value, *, layer, model_version='v1', window='1m', **ctx):
    row = [
        datetime.utcnow().isoformat(timespec='seconds') + 'Z',
        name,
        float(value),
        layer,
        model_version,
        window,
        json.dumps(ctx)
    ]
    with METRICS_CSV.open('a', newline='') as f:
        csv.writer(f).writerow(row)
    return row

# quick smoke test
log_metric('job_success', 1.0, layer='system', window='1d', job='nightly_batch')

['2025-08-29T03:11:03Z',
 'job_success',
 1.0,
 'system',
 'v1',
 '1d',
 '{"job": "nightly_batch"}']

In [4]:
random.seed(42)
y_true,y_pred=[],[]
for t in range(120):
    y=random.random()
    noise=0.05 if t<=80 else 0.15
    pred=min(1.0,max(0.0,y+random.uniform(-noise,noise)))
    y_true.append(y)
    y_pred.append(pred)
    if t>=19:
        window_true=y_true[t-19:t+1]
        window_pred = y_pred[t-19:t+1]
        mae=sum(abs(a-b) for a,b in zip(window_true,window_pred))/20
        row=log_metric('rolling_mae', mae, layer='model', window='20obs', step=t)
        latency=random.randint(60,160) if t<=80 else random.randint(120,300)
        log_metric('p95_latency_ms',latency,layer='system',window='1m',step=t)
print(row)

['2025-08-29T05:37:25Z', 'rolling_mae', 0.07433442759085997, 'model', 'v1', '20obs', '{"step": 119}']


In [ ]:
RULES={'rolling_mae'}